Import Libaries:

pip install numpy pandas tabulate matplotlib tensorflow scikit-learn

In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import tensorflow as tf #Artificial Neural Network
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.ensemble import RandomForestClassifier #Random Forests
from sklearn.linear_model import LogisticRegression #Logistic Regression
from sklearn.model_selection import GridSearchCV #hyperparameters
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('creditcard.csv')

In [ ]:
# features of dataset
credit_card_data.info()

The Class feature (30th) is the label of each data entry i.e (0,1) = (Fraudulent,Legit)

In [3]:
# distribution of 'Class'
credit_card_data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

The distribution of transactions is very unbalanced i.e 284315 fraud vs 492 legit

Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions



So we choose a 1:1 ratio i.e 492 legit : 492 fraudulent transactions, to prevent an invalid accuracy score of the model.

In [4]:
# separating the data for Under-Sampling
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]
legit_sample = legit.sample(n=492, random_state=4)

Concatenating the legit_sample and fraud DataFrames to make  the new, undersampled dataset which is balanced.

In [5]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)
new_dataset['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

Splitting the data into Features (input variable) & Targets (output variable) 

In [6]:
X = new_dataset.drop(columns='Class', axis=1) #input
Y = new_dataset['Class'] #output

Create the training and test datasets

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)
print("Balanced dataset: {} Train Dataset: {} Test Dataset: {}".format(X.shape, X_train.shape, X_test.shape))

Balanced dataset: (984, 30) Train Dataset: (787, 30) Test Dataset: (197, 30)


Model Evaluation and Utilities

In [20]:
logs = []
LRrun = 0
DTrun = 0
RFrun = 0
TFrun = 0

def test(modelstr):
    #Test model on train and test data    
    global X_train_prediction, X_test_prediction
    X_train_prediction = np.round(model.predict(X_train))
    X_test_prediction = np.round(model.predict(X_test))
    results = [0]*11
    results[0] = accuracy_score(Y_train, X_train_prediction)
    results[1] = accuracy_score(Y_test, X_test_prediction)
    results[2] = precision_score(Y_train, X_train_prediction)
    results[3] = precision_score(Y_test, X_test_prediction)
    results[4] = recall_score(Y_train, X_train_prediction)
    results[5] = recall_score(Y_test, X_test_prediction)
    results[6] = f1_score(Y_train, X_train_prediction)
    results[7] = f1_score(Y_test, X_test_prediction)
    results[8] = roc_auc_score(Y_train, X_train_prediction)
    results[9] = roc_auc_score(Y_test, X_test_prediction)
    results[10] = modelstr
    logs.append(results)

Confusion Matrix

In [ ]:
ConfTrain = ConfusionMatrixDisplay.from_predictions(Y_train, X_train_prediction) #Conf Matrix on training data

In [ ]:
ConfTest = ConfusionMatrixDisplay.from_predictions(Y_test, X_test_prediction) #Conf Matrix on test data

Models and Training

In [21]:
# training the Logistic Regression Model with Training Data
# Define parameter grid for Logistic Regression
param_grid = {'C': [0.1, 0.5, 1, 1.5, 1.9], 'penalty': ['l2', 'none']}

# Use GridSearchCV to find the best hyperparameters
model = GridSearchCV(LogisticRegression(random_state=42), param_grid, cv=5, n_jobs=-1)
model.fit(X_train, Y_train)

# Print the best hyperparameters and their corresponding score
print("Best Hyperparameters: ", model.best_params_)
print("Best Mean Cross-validated Score: ", model.best_score_)
LRrun += 1
modelstr = "Logistic Regression " + str(LRrun)
test(modelstr)


Best Hyperparameters:  {'C': 0.1, 'penalty': 'l2'}
Best Mean Cross-validated Score:  0.9263000886882207


In [22]:
# training the Decision Tree on Training Data
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 4, 6, 8, 10, 12],
    'min_samples_split': [2, 4, 6, 8, 10, 12]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Print the best hyperparameters and the corresponding mean cross-validated score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Mean Cross-validated Score:", grid_search.best_score_)

# Use the best model from GridSearchCV for evaluation
model = grid_search.best_estimator_
DTrun += 1
modelstr = "Decision Tree " + str(DTrun)
test(modelstr)

Best Hyperparameters: {'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2}
Best Mean Cross-validated Score: 0.9237926308151254


In [23]:
# training Random Forests on training data
from sklearn.model_selection import GridSearchCV

# Define hyperparameter search space
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(RandomForestClassifier(n_estimators=100, random_state=42), param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Print the best hyperparameters and the corresponding mean cross-validated score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Mean Cross-validated Score:", grid_search.best_score_)

# Use the best model from GridSearchCV for evaluation
model = grid_search.best_estimator_
RFrun += 1
modelstr = "Random Forest " + str(RFrun)
test(modelstr)

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best Mean Cross-validated Score: 0.941554462630009


In [24]:
# create an ANN using tensorflow

# Define the function that creates the Keras model

# Set TensorFlow to use the GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

        
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create a KerasClassifier object for use with GridSearchCV
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=32)

# Define hyperparameter search space
param_grid = {'batch_size': [16, 32],
              'epochs': [100,200,400,800]}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, Y_train)

# Print results
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))

model = grid_search.best_estimator_
TFrun += 1
modelstr = "ANN " + str(TFrun)
test(modelstr)

Epoch 1/100


C:\Users\fazal\AppData\Local\Temp\ipykernel_22712\3149380035.py:25: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=32)


40/40 [==============================] - 1s 4ms/step - loss: 1005.7130 - accuracy: 0.5087
Epoch 2/100
40/40 [==============================] - 0s 3ms/step - loss: 242.3767 - accuracy: 0.4960
Epoch 3/100
40/40 [==============================] - 0s 4ms/step - loss: 135.8602 - accuracy: 0.5390
Epoch 4/100
40/40 [==============================] - 0s 3ms/step - loss: 234.6550 - accuracy: 0.5119
Epoch 5/100
40/40 [==============================] - 0s 3ms/step - loss: 159.1840 - accuracy: 0.5215
Epoch 6/100
40/40 [==============================] - 0s 3ms/step - loss: 257.2694 - accuracy: 0.5135
Epoch 7/100
40/40 [==============================] - 0s 3ms/step - loss: 290.8704 - accuracy: 0.4769
Epoch 8/100
40/40 [==============================] - 0s 3ms/step - loss: 382.2045 - accuracy: 0.5278
Epoch 9/100
40/40 [==============================] - 0s 4ms/step - loss: 446.7666 - accuracy: 0.4897
Epoch 10/100
40/40 [==============================] - 0s 3ms/step - loss: 257.8375 - accuracy: 0.5199


Logging

In [25]:
score_names = []
training_data = []
test_data = []
experiment_name = []

for results in logs:
    # Define the score names, training data values, and test data values as lists
    score_names.extend(["Accuracy", "Precision", "Recall", "F1", "ROC-AUC"])
    # Append the training and test data to their respective lists
    training_data.extend(["{:.5f}".format(results[i]) for i in range(0, 10, 2)])
    test_data.extend(["{:.5f}".format(results[i]) for i in range(1, 10, 2)])
    exp_name = [results[10]]
    # Define the Experiment Name as a list
    exp_name.extend([""] * (4))
    experiment_name.extend(exp_name)

# Combine the lists to create the rows for the table
rows = [[experiment_name[i], score_names[i], training_data[i], test_data[i]] for i in range(len(training_data))]

# Define the header for the table
header = ["Experiment Name", "Score", "Training Data", "Test Data"]

# Print the table
print(tabulate(rows, headers=header, tablefmt="fancy_grid"))

╒═══════════════════════╤═══════════╤═════════════════╤═════════════╕
│ Experiment Name       │ Score     │   Training Data │   Test Data │
╞═══════════════════════╪═══════════╪═════════════════╪═════════════╡
│ Logistic Regression 1 │ Accuracy  │         0.93774 │     0.95939 │
├───────────────────────┼───────────┼─────────────────┼─────────────┤
│                       │ Precision │         0.96748 │     0.97872 │
├───────────────────────┼───────────┼─────────────────┼─────────────┤
│                       │ Recall    │         0.90609 │     0.93878 │
├───────────────────────┼───────────┼─────────────────┼─────────────┤
│                       │ F1        │         0.93578 │     0.95833 │
├───────────────────────┼───────────┼─────────────────┼─────────────┤
│                       │ ROC-AUC   │         0.93778 │     0.95929 │
├───────────────────────┼───────────┼─────────────────┼─────────────┤
│ Decision Tree 1       │ Accuracy  │         0.9352  │     0.93401 │
├───────────────────